In [1]:
import sysconfig

from pyspark.sql import SparkSession

In [22]:
spark = (
    SparkSession
        .builder
        .remote("sc://localhost:15002")
        .appName("hive")
        .enableHiveSupport()
        .config("hive.metastore.uris", "thrift://localhost:9083")
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")
        .getOrCreate()
)

# Load the Spark Connect jar
spark_connect_jar = f"{sysconfig.get_paths()['purelib']}/pyspark/jars/spark-connect_2.13-4.0.1.jar"
spark.addArtifacts(spark_connect_jar)

In [29]:
#spark.sql("CREATE TABLE IF NOT EXISTS students (name VARCHAR(64), age INT, gpa DECIMAL(3, 2)) STORED AS PARQUET LOCATION 'hdfs://hdfs-cluster-namenode-default-0.hdfs-cluster-namenode-default.bigdata.svc.cluster.local/user/hive/warehouse/students'")
#spark.sql("INSERT INTO students (name, age, gpa) VALUES ('Hello world123456', 20, 3.5)")
#spark.sql("REFRESH TABLE students")
#spark.sql("DROP TABLE students")
#spark.sql("clear cache")
spark.sql("INSERT OVERWRITE TABLE students SELECT * FROM students;")
#mydf = spark.sql("select * from students")
#mydf.show()

AnalysisException: [UNSUPPORTED_OVERWRITE.TABLE] Can't overwrite the target that is also being read from. The target table is `spark_catalog`.`default`.`students`. SQLSTATE: 42902

JVM stacktrace:
org.apache.spark.sql.AnalysisException
	at org.apache.spark.sql.errors.QueryCompilationErrors$.cannotOverwriteTableThatIsBeingReadFromError(QueryCompilationErrors.scala:2903)
	at org.apache.spark.sql.execution.command.DDLUtils$.verifyNotReadPath(ddl.scala:1087)
	at org.apache.spark.sql.execution.datasources.DataSourceAnalysis$$anonfun$apply$1.applyOrElse(DataSourceStrategy.scala:240)
	at org.apache.spark.sql.execution.datasources.DataSourceAnalysis$$anonfun$apply$1.applyOrElse(DataSourceStrategy.scala:141)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$resolveOperatorsDownWithPruning$2(AnalysisHelper.scala:200)
	at org.apache.spark.sql.catalyst.trees.CurrentOrigin$.withOrigin(origin.scala:86)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper.$anonfun$reso